<a href="https://colab.research.google.com/github/lesoame/Python_Mundo_1/blob/main/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#instalar o java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [26]:
#Baixar a versão mais recente do spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [27]:
#Deszipar o spark
!tar xf /content/spark-3.5.0-bin-hadoop3.tgz

In [31]:
#Criar variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [29]:
# Instalar a lib findspark que ajuda a localizar o Spark no sistema e importálo como uma biblioteca regular
!pip install -q findspark

In [32]:
#Importar a lib findspark
import findspark
findspark.init()

In [33]:
#Criar spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('sparkcolab')\
        .getOrCreate()

In [34]:
#importar o dataset
from google.colab import files
arquivo = files.upload()

Saving imdb_top_1000.csv to imdb_top_1000.csv


In [35]:
#Criar dataframe
df = spark.read.csv('imdb_top_1000.csv', header=True, inferSchema=True)

In [36]:
df.printSchema()

root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: string (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: string (nullable = true)
 |-- Gross: string (nullable = true)



In [37]:
df.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [41]:
#Selecionar colunas
df_series = df.select('Series_Title', 'Genre', 'Director', 'No_of_Votes')

In [42]:
#mostrar na tela
df_series.show()

+--------------------+--------------------+--------------------+-----------+
|        Series_Title|               Genre|            Director|No_of_Votes|
+--------------------+--------------------+--------------------+-----------+
|The Shawshank Red...|               Drama|      Frank Darabont|    2343110|
|       The Godfather|        Crime, Drama|Francis Ford Coppola|    1620367|
|     The Dark Knight|Action, Crime, Drama|   Christopher Nolan|    2303232|
|The Godfather: Pa...|        Crime, Drama|Francis Ford Coppola|    1129952|
|        12 Angry Men|        Crime, Drama|        Sidney Lumet|     689845|
|The Lord of the R...|Action, Adventure...|       Peter Jackson|    1642758|
|        Pulp Fiction|        Crime, Drama|   Quentin Tarantino|    1826188|
|    Schindler's List|Biography, Drama,...|    Steven Spielberg|    1213505|
|           Inception|Action, Adventure...|   Christopher Nolan|    2067042|
|          Fight Club|               Drama|       David Fincher|    1854740|

In [43]:
#mudar tipo das colunas
df_Votes = df_series.withColumn('Votes', df_series['No_of_Votes'].cast('int')).drop('No_of_Votes')
df_Votes.show()

+--------------------+--------------------+--------------------+-------+
|        Series_Title|               Genre|            Director|  Votes|
+--------------------+--------------------+--------------------+-------+
|The Shawshank Red...|               Drama|      Frank Darabont|2343110|
|       The Godfather|        Crime, Drama|Francis Ford Coppola|1620367|
|     The Dark Knight|Action, Crime, Drama|   Christopher Nolan|2303232|
|The Godfather: Pa...|        Crime, Drama|Francis Ford Coppola|1129952|
|        12 Angry Men|        Crime, Drama|        Sidney Lumet| 689845|
|The Lord of the R...|Action, Adventure...|       Peter Jackson|1642758|
|        Pulp Fiction|        Crime, Drama|   Quentin Tarantino|1826188|
|    Schindler's List|Biography, Drama,...|    Steven Spielberg|1213505|
|           Inception|Action, Adventure...|   Christopher Nolan|2067042|
|          Fight Club|               Drama|       David Fincher|1854740|
|The Lord of the R...|Action, Adventure...|       P

In [47]:
#filtro
df_Votes.filter(df_Votes.Votes > 1500000).show()

+--------------------+--------------------+--------------------+-------+
|        Series_Title|               Genre|            Director|  Votes|
+--------------------+--------------------+--------------------+-------+
|The Shawshank Red...|               Drama|      Frank Darabont|2343110|
|       The Godfather|        Crime, Drama|Francis Ford Coppola|1620367|
|     The Dark Knight|Action, Crime, Drama|   Christopher Nolan|2303232|
|The Lord of the R...|Action, Adventure...|       Peter Jackson|1642758|
|        Pulp Fiction|        Crime, Drama|   Quentin Tarantino|1826188|
|           Inception|Action, Adventure...|   Christopher Nolan|2067042|
|          Fight Club|               Drama|       David Fincher|1854740|
|The Lord of the R...|Action, Adventure...|       Peter Jackson|1661481|
|        Forrest Gump|      Drama, Romance|     Robert Zemeckis|1809221|
|          The Matrix|      Action, Sci-Fi|      Lana Wachowski|1676426|
|        Interstellar|Adventure, Drama,...|   Chris

In [48]:
#pegar max
from pyspark.sql.functions import max
df_max = df_Votes.agg(max('Votes').alias('max_votes')).show()

+---------+
|max_votes|
+---------+
|  2343110|
+---------+



In [49]:
df_max_votes = df_Votes.filter(df_Votes.Votes == 2343110).show()

+--------------------+-----+--------------+-------+
|        Series_Title|Genre|      Director|  Votes|
+--------------------+-----+--------------+-------+
|The Shawshank Red...|Drama|Frank Darabont|2343110|
+--------------------+-----+--------------+-------+



In [52]:
#contar genre
from pyspark.sql.functions import count, col, asc, desc
df_sum = df_Votes.groupBy('genre').count()
df_sum.show(truncate=False)

+-----------------------------+-----+
|genre                        |count|
+-----------------------------+-----+
|Action, Thriller             |3    |
|Animation, Comedy, Fantasy   |1    |
|Crime, Thriller              |3    |
|Action, Adventure, Fantasy   |6    |
|Adventure, Sci-Fi            |2    |
|Drama, Fantasy, History      |1    |
|Animation, Adventure, Fantasy|1    |
|Animation, Biography, Crime  |1    |
|Animation, Adventure, Drama  |7    |
|Action, Adventure, History   |1    |
|Comedy, Crime, Mystery       |1    |
|Adventure, Biography, Crime  |1    |
|Comedy, Drama                |35   |
|Biography, Drama, Sport      |9    |
|Comedy, Drama, Romance       |31   |
|Action, Drama                |5    |
|Drama, Romance, Thriller     |2    |
|Drama, Mystery, Sci-Fi       |5    |
|Comedy, Family, Romance      |1    |
|Adventure, Drama, Fantasy    |6    |
+-----------------------------+-----+
only showing top 20 rows



In [54]:
df_sum.orderBy(col('count').desc()).show(truncate=False)

+----------------------------+-----+
|genre                       |count|
+----------------------------+-----+
|Drama                       |85   |
|Drama, Romance              |37   |
|Comedy, Drama               |35   |
|Comedy, Drama, Romance      |31   |
|Action, Crime, Drama        |30   |
|Crime, Drama, Thriller      |28   |
|Biography, Drama, History   |28   |
|Crime, Drama, Mystery       |27   |
|Crime, Drama                |26   |
|Animation, Adventure, Comedy|24   |
|Action, Adventure, Sci-Fi   |21   |
|Biography, Crime, Drama     |16   |
|Comedy, Crime, Drama        |15   |
|Drama, War                  |15   |
|Action, Adventure, Drama    |14   |
|Drama, Thriller             |14   |
|Comedy                      |13   |
|Biography, Drama            |12   |
|Animation, Action, Adventure|11   |
|Action, Adventure, Comedy   |10   |
+----------------------------+-----+
only showing top 20 rows



In [72]:
#in
df_Votes.select('Series_Title', 'Genre', 'Votes').filter(df_Votes.Genre.isin('Crime, Drama', 'Drama, War', 'Comedy')).show(truncate=False)

+--------------------------------------------------------------------+------------+-------+
|Series_Title                                                        |Genre       |Votes  |
+--------------------------------------------------------------------+------------+-------+
|The Godfather                                                       |Crime, Drama|1620367|
|The Godfather: Part II                                              |Crime, Drama|1129952|
|12 Angry Men                                                        |Crime, Drama|689845 |
|Pulp Fiction                                                        |Crime, Drama|1826188|
|Cidade de Deus                                                      |Crime, Drama|699256 |
|Saving Private Ryan                                                 |Drama, War  |1235804|
|Once Upon a Time in America                                         |Crime, Drama|311365 |
|Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb|Comedy    

In [73]:
#para negar todos os gêneros que estão na lista basta adicionar ~ no começo da função isin
df_Votes.select('Series_Title', 'Genre', 'Votes').filter(~df_Votes.Genre.isin('Crime, Drama', 'Drama, War', 'Comedy')).show(truncate=False)

+-------------------------------------------------+----------------------------+-------+
|Series_Title                                     |Genre                       |Votes  |
+-------------------------------------------------+----------------------------+-------+
|The Shawshank Redemption                         |Drama                       |2343110|
|The Dark Knight                                  |Action, Crime, Drama        |2303232|
|The Lord of the Rings: The Return of the King    |Action, Adventure, Drama    |1642758|
|Schindler's List                                 |Biography, Drama, History   |1213505|
|Inception                                        |Action, Adventure, Sci-Fi   |2067042|
|Fight Club                                       |Drama                       |1854740|
|The Lord of the Rings: The Fellowship of the Ring|Action, Adventure, Drama    |1661481|
|Forrest Gump                                     |Drama, Romance              |1809221|
|Il buono, il brutto,

In [76]:
#usando sql
df_Votes.createOrReplaceTempView('movies')
spark.sql('select Votes, count(*) as qtd from movies GROUP BY Votes ORDER BY qtd desc').show(truncate=False)

+-------+---+
|Votes  |qtd|
+-------+---+
|NULL   |14 |
|65341  |2  |
|959181 |1  |
|939631 |1  |
|244430 |1  |
|1661481|1  |
|1159315|1  |
|425457 |1  |
|211427 |1  |
|206918 |1  |
|43690  |1  |
|78925  |1  |
|918562 |1  |
|241513 |1  |
|1125712|1  |
|50610  |1  |
|582239 |1  |
|131436 |1  |
|35713  |1  |
|27733  |1  |
+-------+---+
only showing top 20 rows



In [ ]:
#criar df reviews